# **Case Study 4: Neural Network and Hyper Parameter Tuning**

### **Installing required packages**

In [39]:
!pip install tensorflow

  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)


## **Importing Libraries**

In [46]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## **Dataset**

In [2]:
df = pd.read_csv('magic_gamma_telescope04_.csv')

In [3]:
df.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [5]:
df.shape

(19020, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19020 entries, 0 to 19019
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   flength   19020 non-null  float64
 1   fwidth    19020 non-null  float64
 2   fsize     19020 non-null  float64
 3   fconc     19020 non-null  float64
 4   fconc1    19020 non-null  float64
 5   fsym      19020 non-null  float64
 6   fm3long   19020 non-null  float64
 7   fm3trans  19020 non-null  float64
 8   falpha    19020 non-null  float64
 9   dist      19020 non-null  float64
 10  class     19020 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.6+ MB


#### Dataset contains no NaN values and all the dependent columns are of numerical type.

In [72]:
df.describe()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


## **Dependent and Independent Features**

In [22]:
X = df.loc[:, df.columns != 'class'].values
y = df.loc[:, 'class'].values

In [23]:
X.shape, y.shape

((19020, 10), (19020,))

In [24]:
X

array([[ 28.7967,  16.0021,   2.6449, ...,  -8.2027,  40.092 ,  81.8828],
       [ 31.6036,  11.7235,   2.5185, ...,  -9.9574,   6.3609, 205.261 ],
       [162.052 , 136.031 ,   4.0612, ..., -45.216 ,  76.96  , 256.788 ],
       ...,
       [ 75.4455,  47.5305,   3.4483, ...,  -9.4662,  30.2987, 256.5166],
       [120.5135,  76.9018,   3.9939, ..., -63.8389,  84.6874, 408.3166],
       [187.1814,  53.0014,   3.2093, ...,  31.4755,  52.731 , 272.3174]])

In [25]:
y

array(['g', 'g', 'g', ..., 'h', 'h', 'h'], dtype=object)

In [30]:
np.unique(y)

array(['g', 'h'], dtype=object)

## **Label Encoding**

In [32]:
le = LabelEncoder()
y = le.fit_transform(y)

In [33]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [34]:
np.unique(y)

array([0, 1])

## **Scaling X value**

In [40]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [41]:
X

array([[0.0743064 , 0.06241507, 0.20804258, ..., 0.51249308, 0.44546667,
        0.1630664 ],
       [0.08281491, 0.04572669, 0.17066824, ..., 0.50794422, 0.07067667,
        0.41267917],
       [0.4782407 , 0.53057937, 0.62681845, ..., 0.41654048, 0.85511111,
        0.51692609],
       ...,
       [0.21571204, 0.18538938, 0.44559432, ..., 0.5092176 , 0.33665222,
        0.51637701],
       [0.35232583, 0.29995007, 0.60691898, ..., 0.36826282, 0.94097111,
        0.82349138],
       [0.55441499, 0.20672824, 0.37492608, ..., 0.61535411, 0.5859    ,
        0.54834441]])

## **Train and Test Data**

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [43]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14265, 10), (4755, 10), (14265,), (4755,))

## **Model Building**

In [163]:
model = Sequential()
model.add(Dense(5, input_shape=(10,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [164]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 5)                 55        
                                                                 
 dense_7 (Dense)             (None, 1)                 6         
                                                                 
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [165]:
early_stop = EarlyStopping(monitor='accuracy', patience=5)

In [166]:
model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stop], validation_data=(X_test, y_test)) 

Epoch 1/100
446/446 [==============================] - 1s 2ms/step - loss: 0.6243 - accuracy: 0.6715 - val_loss: 0.5727 - val_accuracy: 0.7411
Epoch 2/100
446/446 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7596 - val_loss: 0.5065 - val_accuracy: 0.7642
Epoch 3/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4930 - accuracy: 0.7759 - val_loss: 0.4867 - val_accuracy: 0.7851
Epoch 4/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4790 - accuracy: 0.7833 - val_loss: 0.4752 - val_accuracy: 0.7802
Epoch 5/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4716 - accuracy: 0.7870 - val_loss: 0.4707 - val_accuracy: 0.7844
Epoch 6/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4673 - accuracy: 0.7891 - val_loss: 0.4682 - val_accuracy: 0.7853
Epoch 7/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4637 - accuracy: 0.7914 - val_loss: 0.4653 - val_accuracy: 0.7853

## **Model Evaluation**

In [167]:
model.evaluate(X_test, y_test)

149/149 [==============================] - 0s 1ms/step - loss: 0.4582 - accuracy: 0.7819


[0.4582381546497345, 0.7819137573242188]

In [168]:
y_pred = model.predict(X_test).flatten()
y_pred = (y_pred>0.5).astype('int')
y_pred

149/149 [==============================] - 0s 1ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [169]:
confusion_matrix(y_test, y_pred)

array([[2821,  281],
       [ 756,  897]], dtype=int64)

In [170]:
accuracy_score(y_test, y_pred)

0.7819137749737118

In [171]:
f1_score(y_test, y_pred)

0.6336983398092547

#### The initial model gives an accuracy of 78.19% and f1 score of 63.36%

## **Hyper Parameter Tuning**

In [101]:
from keras_tuner.tuners import RandomSearch

In [145]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('Neurons', min_value=10, max_value=30, step=5), 
                    input_shape=(10,), activation='relu'))
    model.add(Dropout(hp.Float('Dropouts', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('Optimizer', values=['adam', 'sgd']), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [146]:
tuner = RandomSearch(build_model,
                     objective='accuracy')
                     #max_trials=5,
                     #executions_per_trial=3)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [147]:
tuner.search_space_summary()

Search space summary
Default search space size: 3
Neurons (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 30, 'step': 5, 'sampling': None}
Dropouts (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
Optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd'], 'ordered': False}


In [148]:
tuner.search(X_train, y_train, epochs=50, batch_size=32, callbacks=[early_stop], validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 34s]
accuracy: 0.7988082766532898

Best accuracy So Far: 0.8397476077079773
Total elapsed time: 00h 03m 22s
INFO:tensorflow:Oracle triggered exit


In [149]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
Neurons: 15
Dropouts: 0.1
Optimizer: adam
Score: 0.8397476077079773
Trial summary
Hyperparameters:
Neurons: 15
Dropouts: 0.30000000000000004
Optimizer: adam
Score: 0.8268489241600037
Trial summary
Hyperparameters:
Neurons: 15
Dropouts: 0.2
Optimizer: adam
Score: 0.8263581991195679
Trial summary
Hyperparameters:
Neurons: 10
Dropouts: 0.4
Optimizer: adam
Score: 0.8060287435849508
Trial summary
Hyperparameters:
Neurons: 30
Dropouts: 0.2
Optimizer: sgd
Score: 0.8052108685175577
Trial summary
Hyperparameters:
Neurons: 20
Dropouts: 0.2
Optimizer: sgd
Score: 0.8043930331865946
Trial summary
Hyperparameters:
Neurons: 25
Dropouts: 0.2
Optimizer: sgd
Score: 0.8007711172103882
Trial summary
Hyperparameters:
Neurons: 10
Dropouts: 0.1
Optimizer: sgd
Score: 0.7990185817082723
Trial summary
Hyperparameters:
Neurons: 30
Dropouts: 0.4
Optimizer: sgd
Score: 0.7988082766532898
Trial summary
Hyperparameters

## **Best Model**

Neurons: 15  
Dropouts: 0.1  
Optimizer: adam  

In [175]:
final_model = Sequential()
final_model.add(Dense(15, input_shape=(10,), activation='relu'))
final_model.add(Dropout(0.1))
final_model.add(Dense(1, activation='sigmoid'))

final_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [176]:
final_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 15)                165       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 16        
                                                                 
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [177]:
final_model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[early_stop], validation_data=(X_test, y_test)) 

Epoch 1/100
446/446 [==============================] - 2s 3ms/step - loss: 0.5895 - accuracy: 0.6971 - val_loss: 0.5138 - val_accuracy: 0.7613
Epoch 2/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4950 - accuracy: 0.7750 - val_loss: 0.4737 - val_accuracy: 0.7834
Epoch 3/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4717 - accuracy: 0.7877 - val_loss: 0.4586 - val_accuracy: 0.7937
Epoch 4/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4559 - accuracy: 0.7921 - val_loss: 0.4483 - val_accuracy: 0.7958
Epoch 5/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4444 - accuracy: 0.7962 - val_loss: 0.4399 - val_accuracy: 0.7966
Epoch 6/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4394 - accuracy: 0.7973 - val_loss: 0.4362 - val_accuracy: 0.8004
Epoch 7/100
446/446 [==============================] - 1s 2ms/step - loss: 0.4357 - accuracy: 0.7996 - val_loss: 0.4307 - val_accuracy: 0.7989

## **Model Evaluation**

In [178]:
final_model.evaluate(X_test, y_test)

149/149 [==============================] - 0s 2ms/step - loss: 0.3578 - accuracy: 0.8530


[0.3578225374221802, 0.852996826171875]

In [179]:
y_pred = final_model.predict(X_test).flatten()
y_pred = (y_pred>0.5).astype('int')
y_pred

149/149 [==============================] - 0s 1ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [180]:
confusion_matrix(y_test, y_pred)

array([[2883,  219],
       [ 480, 1173]], dtype=int64)

In [181]:
accuracy_score(y_test, y_pred)

0.8529968454258675

In [182]:
f1_score(y_test, y_pred)

0.7704433497536947

#### The best model from the hyper tuned models gives an accuracy of 85.29% and f1 score of 77.04%